# PieceMatcher sample

Use the `PieceMatcher`


## Import


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from loguru import logger as lg
from rich import get_console
from rich import print as rprint
from rich.console import Console

# some magic to make rich work in jupyter
# https://github.com/Textualize/rich/issues/3483
# enable it for every cell output with %load_ext rich
console: Console = get_console()
console.is_jupyter = False

In [ ]:
from snap_fit.config.aruco.aruco_board_config import ArucoBoardConfig
from snap_fit.config.aruco.aruco_detector_config import ArucoDetectorConfig
from snap_fit.data_models.segment_id import SegmentId
from snap_fit.image.segment_matcher import SegmentMatcher
from snap_fit.params.snap_fit_params import get_snap_fit_paths
from snap_fit.puzzle.piece_matcher import PieceMatcher
from snap_fit.puzzle.sheet_aruco import SheetAruco
from snap_fit.puzzle.sheet_manager import SheetManager


## Params and config


In [ ]:
sf_paths = get_snap_fit_paths()
rprint(sf_paths)

### Load oca data


In [ ]:
# 1. Configure ArUco Board and Detector
# Using defaults which match the printed board used for 'data/oca'
board_config = ArucoBoardConfig(markers_x=5, markers_y=7)
detector_config = ArucoDetectorConfig(board=board_config)

# 2. Initialize SheetAruco helper
# crop_margin is automatically calculated from the detector configuration
sheet_aruco = SheetAruco(detector_config)

# 3. Define the loader function
# SheetAruco.load_sheet handles loading, rectification, and Sheet creation
aruco_loader = sheet_aruco.load_sheet

# 4. define base folder
paths = get_snap_fit_paths()
data_dir = paths.data_fol / "oca"
lg.info(f"Loading data from {data_dir}")

# 5. instantiate manager and load
manager = SheetManager()
manager.add_sheets(folder_path=data_dir, pattern="*.jpg", loader_func=aruco_loader)

## Piece match


In [ ]:
# 1. Initialize the PieceMatcher
matcher = PieceMatcher(manager)

# 2. Run matching for all segments
# This will compare every segment with segments from other pieces
matcher.match_all()

# 3. Get and display the top 5 matches
top_matches = matcher.get_top_matches(5)
rprint("[bold green]Top 5 Matches:[/bold green]")
for i, res in enumerate(top_matches, 1):
    rprint(f"{i}. {res.seg_id1} <-> {res.seg_id2} | Similarity: {res.similarity:.4f}")

# 4. Demonstrate querying matches for a specific piece
# Let's pick the first piece from the first sheet
all_ids = manager.get_segment_ids_all()
if all_ids:
    sample_id = all_ids[0]
    piece_matches = matcher.get_matches_for_piece(
        sample_id.sheet_id, sample_id.piece_id
    )

    rprint(
        f"\n[bold blue]Matches for piece {sample_id.sheet_id}:{sample_id.piece_id}:[/bold blue]"
    )
    # Show top 3 matches for this piece
    piece_matches.sort(key=lambda x: x.similarity)
    for res in piece_matches[:3]:
        other = res.get_other(
            SegmentId(
                sheet_id=sample_id.sheet_id,
                piece_id=sample_id.piece_id,
                edge_pos=res.seg_id1.edge_pos
                if res.seg_id1.piece_id == sample_id.piece_id
                else res.seg_id2.edge_pos,
            )
        )
        # Note: get_other needs the exact SegmentId.
        # Let's simplify the display:
        rprint(
            f"  - {res.seg_id1} <-> {res.seg_id2} | Similarity: {res.similarity:.4f}"
        )


In [ ]:
# 5. Visualize the best match
import matplotlib.pyplot as plt

if top_matches:
    best = top_matches[0]
    seg1 = manager.get_segment(best.seg_id1)
    seg2 = manager.get_segment(best.seg_id2)

    if seg1 and seg2:
        rprint(f"Visualizing best match: {best.seg_id1} and {best.seg_id2}")
        matcher_viz = SegmentMatcher(seg1, seg2)
        matcher_viz.compute_similarity()

        # Plot the segments
        plt.figure(figsize=(10, 6))

        # Segment 2 (target)
        p2 = seg2.points.reshape(-1, 2)
        plt.plot(p2[:, 0], p2[:, 1], "b-", label=f"Target: {best.seg_id2}", linewidth=2)

        # Segment 1 (transformed)
        p1_trans = matcher_viz.s1_points_transformed.reshape(-1, 2)
        plt.plot(
            p1_trans[:, 0],
            p1_trans[:, 1],
            "r--",
            label=f"Transformed: {best.seg_id1}",
            linewidth=2,
        )

        plt.legend()
        plt.title(f"Best Match (Similarity: {best.similarity:.4f})")
        plt.gca().invert_yaxis()  # Images have y-axis pointing down
        plt.axis("equal")
        plt.show()
